In [1]:
import os, random
import numpy as np
import torch
import dataset_loader

import torch
import torch.nn as nn
import torch.nn.functional as F

# Lab exercises 7

The goal of this lab exercise is to help you learn how to use Pytorch, which you will need to use for the project.

It is important the you read the documentation to understand how to use Pytorch functions, what kind of transformation they apply etc. You have to take time to read it carefully to understand what you are doing.

- https://pytorch.org/docs/stable/nn.html
- https://pytorch.org/docs/stable/torch.html

Each time you use a function, check the manual, even if you think that you know how it works. If you don't read it, you are just making stup\*d decisions.

# 1. Pytorch basics

Instead of manipulating numpy arrays, we will manipulate pytorch tensors.
A lot of things are defined in the same way, except the you can use autograd!

Note that when using pytorch and the autograd mechanism, you want to avoid in-place operations, for reason I didn't have time cover in the course, sorry. :(
The only time you will need in place operation in this course is for parameter initialization.
It is easy to identify in place operations: their function name ends with an underscore!

In [2]:
# create a tensor of shape (2, 4) containing random values.
# by default, it will be a float tensor and will not ask for gradient

t = torch.rand(2, 4)

# you can also create a tensor full of 0 or 1
t_zeros = torch.zeros(2, 4)
t_ones = torch.ones(2, 4)

print(t)
print(t_zeros)
print(t_ones)

tensor([[0.0978, 0.3887, 0.4514, 0.9922],
        [0.6540, 0.9285, 0.8338, 0.3057]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]])


In [3]:
# there also exists *_like functions that creates a tensor with exactly
# the same properties as its argument (shape, gradient requirement, type, etc)

t2 = torch.rand_like(t)
t2_zeros = torch.zeros_like(t)
t2_ones = torch.ones_like(t)

print(t2)
print(t2_zeros)
print(t2_ones)

tensor([[0.4590, 0.7337, 0.9364, 0.3284],
        [0.5707, 0.1305, 0.8040, 0.2844]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]])


In [4]:
# you can also create a tensor of long values (i.e. integers),
# which will be usefull to represent labels :)

t_zeros_long = torch.zeros(10, dtype=torch.long)
print(t_zeros_long)

print(t_zeros_long.dtype, t_zeros.dtype)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
torch.int64 torch.float32


In [5]:
# you can also initialize the tensor with values
t_long = torch.LongTensor([0,1,10,20])
print(t_long)

tensor([ 0,  1, 10, 20])


In [6]:
# now, let's turn to the serious stuff: gradient computation! :)

t = torch.rand(2, 10)

# by default no gradient will be required for t :(
print(t.requires_grad)

# so we ask for it explicitly (note the underscore: in place operation!)
t.requires_grad_(True)
print(t.requires_grad)

# We can also set this to true at creation
t = torch.rand(2, 10, requires_grad=True)
print(t.requires_grad)

# now, let's do a stupid operation and compute the gradient
# this sum over all element of t,
# it return a tensor with a single value
z = t.sum()
print(z.shape, z.requires_grad)

# backpropagation!
z.backward()

# print the gradient of t,
# it should be a vector full of 1, do you understand why?
print(t.grad)

False
True
True
torch.Size([]) True
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [7]:
# if I call backward a second time, it will accumulate the gradient
# so it will be a tensort full of 2
z.backward()
print(t.grad)

# we can reset the gradient of the tensor to zero
# => note the in-place operation
t.grad.zero_()
print(t.grad)

# and if we backprop again, it will be a tensor full of one again
z.backward()
print(t.grad)

# and this highlight one of the major source of bug in pytorch:
# Do not forget to reset your gradients!

tensor([[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [8]:
# similarly to the previous lab exercise,
# the parameter of your network must be encapsulated in a Parameter object:
# https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html
# You should understand why :)
#
# however, Pytorch comes with a lot of modules already made!
# they are in the torch.nn that we often just rename as nn

# https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear
linear = nn.Linear(10, 20)

# parameter of the linear transformation:
# projection matrix W and bias
# look, its a parameter object!
print(type(linear.weight))
print(type(linear.bias))

# the Linear class is not a subtype of parameter
# but of Module, which represent a network part.
print(type(linear), isinstance(linear, nn.Module))

# so remember that we often want to compute values on batches.
# in pytorch, datapoints will be rows instead of columns
# (contrary to the two previous lab exercises with numpy!)
# So, for example, we can create a batch with two random elements,
# each one of size 10, i.e. the input shape is (2, 10)
t_inputs = torch.rand(2, 10)
print(t_inputs.shape)

# compute the hidden representation after the linear transformation!
# note that we use the object as a function for this
t_outputs = linear(t_inputs)
print(t_outputs.shape)

<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.nn.modules.linear.Linear'> True
torch.Size([2, 10])
torch.Size([2, 20])


# Data loading and conversion

In [9]:
# Download mnist dataset 
if("mnist.pkl.gz" not in os.listdir(".")):
    # this link doesn't work any more,
    # seach on google for the file "mnist.pkl.gz"
    # and download it
    !wget https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz

# if you have it somewhere else, you can comment the lines above
# and overwrite the path below
mnist_path = "./mnist.pkl.gz"
# load the 3 splits
train_data, dev_data, test_data = dataset_loader.load_mnist(mnist_path)

def build_torch_inputs(data):
    x, y = data
    ret = list()
    
    for i in range(x.shape[0]):
        input_tensor = torch.from_numpy(x[i]).reshape(1, -1)
        output_value = int(y[i])
        
        ret.append({
            "input_tensor": input_tensor,
            "output_value": output_value
        })
        
    return ret
        
train_data = build_torch_inputs(train_data)
dev_data = build_torch_inputs(dev_data)
test_data = build_torch_inputs(test_data)

In [10]:
# train_data is a list,
# each element is a dictionnary with two keys:
# - input_tensor: the input image as a row vector
# - output_value: the gold label

print(train_data[10]["input_tensor"].shape)
print(train_data[10]["output_value"])

torch.Size([1, 784])
1


In [11]:
# Instead of computing the loss on a single input or on the full dataset,
# it is more common to compute it on a subset of the data, called a batch or minibatch.
# For example, if the we use a batch of size 10,
# the input of the network will be a tensor of shape (10, 784)
# where each row is a single input.
#
# In the data, we already transformed each input into a row vector,
# so we only need to concatenate them.
#
# Here we who an example, of course this is done dynamically during training

# Constructing a batched input
batch_size = 10
first_element = 20 # index in the training set of the first element of the batch

# the cat() function concatenates a list of tensor along a dimension
batch_input = torch.cat(
    [
        data["input_tensor"]
        for data in train_data[first_element:first_element + batch_size]
    ],
    # we want to concatenate on the batch dimension,
    # i.e. the first dimension
    dim=0
)
print(batch_input.shape)  # batch of ten flat images (10, 784)

torch.Size([10, 784])


In [12]:
# just a helper function
def build_batch(data):
    batch_inputs = torch.cat(
        [data["input_tensor"] for data in data],
        dim=0
    )

    labels = torch.LongTensor([data["output_value"] for data in data ])
    
    return batch_inputs, labels

# Network definition and training

In [13]:
# A network network is a class extending nn.Module
class MLPClassifier(nn.Module):
    # constructor, you can define any argument you need
    # to parameterize your network
    def __init__(self, input_dim, hidden_dim, output_dim):
        # you must always call the parent constructor,
        # other it will fail when you will run the network :)
        super().__init__()
        
        # Create the projections:
        # - the first one project from input to hidden space
        # - the second one from hidden space to output space (i.e. logits, weights of each class)
        # Note that if you want to use list or dictionnaries instead of directly
        # setting attributes of the object, you need to use special containers:
        # https://pytorch.org/docs/stable/nn.html#containers
        # do you understand why?
        self.hidden_proj = nn.Linear(input_dim, hidden_dim)
        self.output_proj = nn.Linear(hidden_dim, output_dim)
                
        # custom initialization
        # note that:
        # - we encapsulate in torch.no_grad() to disable autograd here
        # - we use inplace functions (i.e. with an underscore at the end)
        with torch.no_grad():
            torch.nn.init.kaiming_uniform_(self.hidden_proj.weight.data)
            torch.nn.init.kaiming_uniform_(self.output_proj.weight.data)
            
            self.hidden_proj.bias.zero_()
            self.output_proj.bias.zero_()
        
    # the forward function is the one that will be called to compute outputs.
    # note that we never call it directly:
    # we will use the object as a function, as with linear layers
    def forward(self, inputs):
        # first proj
        z = self.hidden_proj(inputs)
        # apply relu
        z = torch.relu(z)

        # apply output proj and return
        return self.output_proj(z)

In [14]:
# Example of a training loop! :)

# hyper-parameters
n_epochs = 10
batch_size = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Build the network
network = MLPClassifier(784, 200, 10)

network.to(device)

# Build the optimizer, i.e. the object that will update parameters
# using the gradient information.
# 
# SGD is standard gradient descent, but there are many alternative!
# https://pytorch.org/docs/stable/optim.html
#
# The first argument of an optimizer is the set of parameters if will update,
# we can use network.parameters() to get all the parameters of our network

# set momentum=0 for standard gradient descent
optimizer = torch.optim.SGD(network.parameters(), lr=1e-3, momentum=0.9)

# Adam is a very good alternative.
#optimizer = torch.optim.Adam(network.parameters())

def train_loop(network, optimizer, train_data, dev_data, n_epochs, batch_size, gradient_clip_value = None):
    for epoch in range(n_epochs):
        print("%i / %i" % (epoch+1, n_epochs))
        
        # shuffle the dataset
        # its a good practice to do this at the beginning of each epoch
        random.shuffle(train_data)
        
        # pass the network in training mode,
        # i.e. dropout will be applied if the dropout module is called
        network.train()
        
        for first_element in range(0, len(train_data), batch_size):
            # IMPORTANT
            # as gradient is accumulated, we need to set all gradients to 0
            # there are several ways of doing that,
            # the simplest is to call optimizer.zero_grad()
            # that set all parameters tracked by the optimizer to 0
            optimizer.zero_grad()
            
            # build our batched input
            batch_input, labels = build_batch(train_data[first_element:first_element + batch_size])
            
            batch_input = batch_input.to(device)
            labels = labels.to(device)
            
            # compute the output weights/logits
            logits = network(batch_input)
            
            # compute the loss
            # https://pytorch.org/docs/stable/nn.functional.html#cross-entropy
            # the torch.nn.functional packages (renamed as F) contains many
            # useful functions that are not network subpart (neither parameters or modules)
            loss = F.cross_entropy(logits, labels)
            
            # compute the gradient
            loss.backward()
            
            # added gradient clipping
            if gradient_clip_value: torch.nn.utils.clip_grad_value_(network.parameters(), gradient_clip_value)

            # update parameters wrt to gradient information!
            optimizer.step()
            
        # at the end of each epoch we evaluate on dev
        # eval on dev data
        n_correct = 0
        # disable auto-grad as we don't need that during evaluation
        # this speed things a little bit + use less memory
        with torch.no_grad(): 
            # pass network in eval mode,
            # i.e. if the dropout module is called,
            # it won't be applied
            network.eval()
            
            for first_element in range(0, len(dev_data), batch_size):
                optimizer.zero_grad()

                batch_input, labels = build_batch(dev_data[first_element:first_element + batch_size])

                batch_input = batch_input.to(device)
                labels = labels.to(device)

                logits = network(batch_input)
                
                # logits is a tensor of shape (batch dim, n labels),
                # to compute the prediction we just compute the argmax
                # along the label dimension
                prediction = logits.argmax(dim=1)
                
                # compare prediction to gold and add to the counter
                # Be carefull: the .item() is used to get a float value
                # instead of a pytorch tensor
                n_correct += (prediction == labels).sum().item()
            
        print("Dev acc: %.2f" % (100 * n_correct / len(dev_data)))
        
train_loop(network, optimizer, train_data, dev_data, n_epochs, batch_size)

Using device: cuda
1 / 10
Dev acc: 92.89
2 / 10
Dev acc: 94.27
3 / 10
Dev acc: 95.35
4 / 10
Dev acc: 95.96
5 / 10
Dev acc: 96.22
6 / 10
Dev acc: 96.76
7 / 10
Dev acc: 96.86
8 / 10
Dev acc: 97.03
9 / 10
Dev acc: 97.20
10 / 10
Dev acc: 97.24


# TODO

The goal of this lab exercise is that you play a little bit with the code above so you can learn how to use pytorch.
I list here a sequence of things that you should be able to implement.

It is really important that you learn how to do that, it will be important for the project.
Of course you need to create the network variant and test it. :)

### 1. Regularization

You can try two types of regularization (they can be combined together):

- weight decay: it is a parameter of the optimizer
- dropout

For dropout, you need to create a dropout layer as part of your network. :)
It will be automatically enabled/disabled when you call network.train()/.eval().
https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html#torch.nn.Dropout

In [15]:
t = torch.ones(2, 4)
print(t)

dropout = nn.Dropout(0.5)

# activate train mode
dropout.train()
t2 = dropout(t)

print(t2)

dropout.eval()
t3 = dropout(t)
print(t3)

# WARNING => of course you don't directly call these functions in the dropout object,
# but instead you call the one of the network that will recursively call it to all
# its module attributes!

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[2., 0., 0., 0.],
        [0., 2., 2., 2.]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]])


A commong trick for training neural networks is gradient clipping: if the norm of the gradient is too big, we rescale the gradient. This trick can be used to prevent exploding gradients and also to make "too big steps" in the wrong direction due the use of approximate gradient computation in SGD

In [ ]:
batch_loss.backward()  # compute gradient
torch.nn.utils.clip_grad_value_(network.parameters(), 5.)  # clip gradient if its norm exceed 5
optimizer.step()  # update parameters

In [16]:
class MLPClassifierWithDropout(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.5):
        super().__init__()

        self.hidden_proj = nn.Linear(input_dim, hidden_dim)
        self.output_proj = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(p=dropout_prob)

        with torch.no_grad():
            torch.nn.init.kaiming_uniform_(self.hidden_proj.weight.data, nonlinearity='relu')
            torch.nn.init.xavier_uniform_(self.output_proj.weight.data)

            self.hidden_proj.bias.zero_()
            self.output_proj.bias.zero_()

    def forward(self, inputs):
        z = self.hidden_proj(inputs)
        z = torch.relu(z)
        z = self.dropout(z)
        return self.output_proj(z)

In [17]:
network = MLPClassifierWithDropout(784, 200, 10)

network.to(device)

weight_decay_val = 1e-4

optimizer = torch.optim.SGD(network.parameters(), lr=1e-3, momentum=0.9, weight_decay=weight_decay_val)

train_loop(network, optimizer, train_data, dev_data, n_epochs, batch_size, gradient_clip_value=1.0)

1 / 10
Dev acc: 92.57
2 / 10
Dev acc: 94.35
3 / 10
Dev acc: 95.23
4 / 10
Dev acc: 95.88
5 / 10
Dev acc: 96.19
6 / 10
Dev acc: 96.56
7 / 10
Dev acc: 96.66
8 / 10
Dev acc: 96.90
9 / 10
Dev acc: 96.98
10 / 10
Dev acc: 97.19


### 2. Deeper network

The second exercise will be to create a deep network!

We will explore 2 different ways of doing that.

**(1)** The most simple technique is to build a list of linear projection in the constructor and set it as an attribute of the network. But **warning**: you should not use a Python list directly, you must instead use a nn.ModuleList(). Luckily, it works as a list: you can append objects and loops of the content. You will also need to update the initialization (to do a loop over all layers!) and the forward pass.

In [32]:
# Example 1
class DeepMLPClassifier1(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_hidden_layers, dropout_prob=0.5):
        super().__init__()

        self.hidden_layers = nn.ModuleList()

        if n_hidden_layers > 0:
            self.hidden_layers.append(nn.Linear(input_dim, hidden_dim))
            for _ in range(n_hidden_layers - 1):
                self.hidden_layers.append(nn.Linear(hidden_dim, hidden_dim))
        else:
            pass

        d_out_proj_in = hidden_dim if n_hidden_layers > 0 else input_dim

        self.output_proj = nn.Linear(d_out_proj_in, output_dim)
        self.dropout = nn.Dropout(dropout_prob)

        with torch.no_grad():
            for layer in self.hidden_layers:
                torch.nn.init.kaiming_uniform_(layer.weight.data, nonlinearity='relu')
                layer.bias.zero_()

            torch.nn.init.kaiming_uniform_(self.output_proj.weight.data)
            self.output_proj.bias.zero_()

    def forward(self, inputs):
        z = inputs

        for layer in self.hidden_layers:
            z = layer(z)
            z = F.relu(z)
            z = self.dropout(z)

        output = self.output_proj(z)
        return output
    
network = DeepMLPClassifier1(784, 200, 10, 3)

batch = torch.rand(10, 784)
output = network(batch)
print(output.shape)

torch.Size([10, 10])


In [33]:
network.to(device)

weight_decay_val = 1e-4

optimizer = torch.optim.SGD(network.parameters(), lr=1e-3, momentum=0.9, weight_decay=weight_decay_val)

train_loop(network, optimizer, train_data, dev_data, n_epochs, batch_size, gradient_clip_value=1.0)

1 / 10
Dev acc: 92.63
2 / 10
Dev acc: 94.62
3 / 10
Dev acc: 95.54
4 / 10
Dev acc: 96.09
5 / 10
Dev acc: 96.18
6 / 10
Dev acc: 96.40
7 / 10
Dev acc: 96.67
8 / 10
Dev acc: 96.80
9 / 10
Dev acc: 96.86
10 / 10
Dev acc: 96.85


**(2)** The second technique is based on a nn.Sequential() object: https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html#torch.nn.Sequential

The idea behind a Sequential() object is that it is a list of sub-modules. When you call the object, it will just execute one module after the other, passing as input of the next one the result of the previous one.

Here is an example on how to use this to construct a single projection with non-linearity and dropout:

In [34]:
# Example of usage: we define a projection as a Sequential object
seq = nn.Sequential(
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Dropout(0.5)  
)

# batched input
inputs = torch.rand(3, 10)

# will call successively the 3 subnetworks,
# i.e. it will apply linear transformation,
# then relu and then dropout
outputs = seq(inputs)
print(outputs.shape)

torch.Size([3, 5])


Unfortunately, it is a little bit more difficult to create than nn.Sequential() because it doesn't have an append() method... but you can use list comprehension + transform the list as a sequence of argument to the constructor.

In [35]:
# the list is a single argument
print([1, 2, 3])

# here we call print with 3 different arguments,
# notice how the output is different
print(1, 2, 3)

# so, how do we call a function by passing the values
# from a list as separate argument?
# Well, like this:
print(*[1, 2, 3])

# notive that this last output is similar to the second one,
# and different from the first! :)

[1, 2, 3]
1 2 3
1 2 3


In [36]:
# Example 2
class DeepMLPClassifier2(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_hidden_layers, dropout_prob=0.5):
        super().__init__()

        layers = [] # Use a temporary Python list to build the sequence

        if n_hidden_layers > 0:
            # First hidden block: input_dim -> hidden_dim
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_prob))

            # Subsequent hidden blocks: hidden_dim -> hidden_dim
            for _ in range(n_hidden_layers - 1):
                layers.append(nn.Linear(hidden_dim, hidden_dim))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(dropout_prob))

        # Create the Sequential container by unpacking the list
        # If layers is empty (n_hidden_layers=0), Sequential is an identity operation
        self.hidden_network = nn.Sequential(*layers)

        # Determine the input dimension for the output layer
        d_out_proj_in = hidden_dim if n_hidden_layers > 0 else input_dim

        self.output_proj = nn.Linear(d_out_proj_in, output_dim)

        # Initialization
        with torch.no_grad():
            # Initialize Linear layers within the Sequential block
            for module in self.hidden_network.modules(): # Iterate through all modules
                if isinstance(module, nn.Linear):
                    torch.nn.init.kaiming_uniform_(module.weight.data, nonlinearity='relu')
                    module.bias.zero_()

            # Initialize the final output layer
            torch.nn.init.kaiming_uniform_(self.output_proj.weight.data)
            self.output_proj.bias.zero_()


    def forward(self, inputs):
        # Pass inputs through the entire hidden network sequence
        z = self.hidden_network(inputs)
        # Apply the final output projection
        output = self.output_proj(z)
        return output
    
network = DeepMLPClassifier2(784, 200, 10, 3)

batch = torch.rand(7, 784)
output = network(batch)
print(output.shape)

torch.Size([7, 10])


In [37]:
network.to(device)

weight_decay_val = 1e-4

optimizer = torch.optim.SGD(network.parameters(), lr=1e-3, momentum=0.9, weight_decay=weight_decay_val)

train_loop(network, optimizer, train_data, dev_data, n_epochs, batch_size, gradient_clip_value=1.0)

1 / 10
Dev acc: 92.43
2 / 10
Dev acc: 94.46
3 / 10
Dev acc: 95.43
4 / 10
Dev acc: 95.78
5 / 10
Dev acc: 96.14
6 / 10
Dev acc: 96.31
7 / 10
Dev acc: 96.64
8 / 10
Dev acc: 96.67
9 / 10
Dev acc: 96.71
10 / 10
Dev acc: 96.95


### 3. Custom module

Now, instead of using the Linear layer, we are going to implement a custom submodule that will:

- apply an linear/affine transformation
- apply a non-linearity
- apply dropout

However, we will do that **wihout** nn.Linear(). Remember that matrix multiplication is done with operator @.

Instead, you need to use you own parameters for the projection matrix and bias.
In the course example and in the previous lab, we use a projection defined as follows: Ax + b where x is the input, A the projection matrix and b the bias vector. However, this assume that the input is a column vector, or, if batched, a matrix where each input is a column of x.

In Pytorch, we use a different format: a single input is a row vector, and a batched input is a matrix where each row in a different input data. So first, let's think a little bit:

- How is the linear projection defined in this case? (no batch, just a single row vector x as input)
- what is the shape of A? of b?
- in the case of a batched input, you need to be careful so that broadcasting is applied correctly. Think about this and what is implies for parameters shape.

To create a parameter in the constructor, you can do: self.whatever = nn.parameter.Parameter(torch.empty(..., ...))

it create a tensor that is unitialized!


**ANSWERS**

xA + b with shapes:

- x: (1, input dim)
- A: (input dim, output dim)
- b: (1, outputdim)


In [38]:
import math

class ManualLinearReLUDropout(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_prob=0.5):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.weight = nn.Parameter(torch.empty(input_dim, output_dim))
        self.bias = nn.Parameter(torch.empty(output_dim))

        self.dropout = nn.Dropout(dropout_prob)

        with torch.no_grad():
            nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5)) # a=sqrt(5) is default for nn.Linear
            nn.init.zeros_(self.bias)

    def forward(self, inputs):
        affine_output = inputs @ self.weight + self.bias
        activated_output = F.relu(affine_output)
        output = self.dropout(activated_output)

        return output


In [41]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_hidden_layers, dropout_prob=0.5):
        super().__init__()

        self.hidden_layers = nn.ModuleList()

        current_dim = input_dim
        if n_hidden_layers > 0:
            for i in range(n_hidden_layers):
                layer = ManualLinearReLUDropout(
                    input_dim=current_dim,
                    output_dim=hidden_dim,
                    dropout_prob=dropout_prob
                )
                self.hidden_layers.append(layer)
                current_dim = hidden_dim
                
        self.output_proj = nn.Linear(current_dim, output_dim)

        with torch.no_grad():
            nn.init.xavier_uniform_(self.output_proj.weight.data)
            self.output_proj.bias.zero_()

    def forward(self, inputs):
        z = inputs

        for layer in self.hidden_layers:
            z = layer(z)

        output = self.output_proj(z)
        return output
    
network = MLPClassifier(784, 200, 10, 3)

# small check that must pass,
# but you should also train it correctly to see if results improve!
batch = torch.rand(10, 784)
output = network(batch)
print(output.shape)

torch.Size([10, 10])


In [42]:
network.to(device)

weight_decay_val = 1e-4

optimizer = torch.optim.SGD(network.parameters(), lr=1e-3, momentum=0.9, weight_decay=weight_decay_val)

train_loop(network, optimizer, train_data, dev_data, n_epochs, batch_size, gradient_clip_value=1.0)

1 / 10
Dev acc: 91.89
2 / 10
Dev acc: 94.64
3 / 10
Dev acc: 95.89
4 / 10
Dev acc: 96.50
5 / 10
Dev acc: 96.55
6 / 10
Dev acc: 96.84
7 / 10
Dev acc: 97.10
8 / 10
Dev acc: 97.32
9 / 10
Dev acc: 97.22
10 / 10
Dev acc: 97.30


###  4. Convolutional Neural Network

Build a network that relies on a CNN instead of a MLP to classify MNIST images
(you can still have a single layer MLP on top of convolutions, after pooling!).
Note that this will requires you to reshape the input images!

https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MNIST_CNN(nn.Module):
    def __init__(self, output_dim=10):
        super().__init__()

        self.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=16,
            kernel_size=5,
            stride=1,
            padding=2
        )
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=5,
            stride=1,
            padding=2
        )
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flattened_size = 32 * 7 * 7

        self.fc = nn.Linear(self.flattened_size, output_dim)

    def forward(self, x):
        x = x.reshape(-1, 1, 28, 28)

        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = torch.flatten(x, start_dim=1)
        logits = self.fc(x)

        return logits

    def _initialize_weights(self):
        with torch.no_grad():
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.normal_(m.weight, 0, 0.01)
                    nn.init.constant_(m.bias, 0)

network = MNIST_CNN(output_dim=10)

batch = torch.rand(10, 784)
output = network(batch)
print(output.shape)

torch.Size([10, 10])


In [44]:
network.to(device)

weight_decay_val = 1e-4

optimizer = torch.optim.SGD(network.parameters(), lr=1e-3, momentum=0.9, weight_decay=weight_decay_val)

train_loop(network, optimizer, train_data, dev_data, n_epochs, batch_size, gradient_clip_value=1.0)

1 / 10
Dev acc: 97.34
2 / 10
Dev acc: 98.19
3 / 10
Dev acc: 98.33
4 / 10
Dev acc: 98.58
5 / 10
Dev acc: 98.42
6 / 10
Dev acc: 98.65
7 / 10
Dev acc: 98.79
8 / 10
Dev acc: 98.67
9 / 10
Dev acc: 98.63
10 / 10
Dev acc: 98.71
